# ME314 Homework 7

###Submission instructions

Deliverables that should be included with your submission are shown in **bold** at the end of each problem statement and the corresponding supplemental material. **Your homework will be graded IFF you submit a single PDF, .mp4 videos of animations when requested and a link to a Google colab file that meet all the requirements outlined below.**

- List the names of students you've collaborated with on this homework assignment.
- Include all of your code (and handwritten solutions when applicable) used to complete the problems.
- Highlight your answers (i.e. **bold** and outline the answers) for handwritten or markdown questions and include simplified code outputs (e.g. .simplify()) for python questions.
- Enable Google Colab permission for viewing 
 * Click Share in the upper right corner
 * Under "Get Link" click "Share with..." or "Change" 
 * Then make sure it says "Anyone with Link" and "Editor" under the dropdown menu
- Make sure all cells are run before submitting (i.e. check the permission by running your code in a private mode)
 * Please don't make changes to your file after submitting, so we can grade it!
- Submit a link to your Google Colab file that has been run (before the submission deadline) and don't edit it afterwards!

**NOTE:** This Jupyter Notebook file serves as a template for you to start homework. Make sure you first copy this template to your own Google driver (click "File" -> "Save a copy in Drive"), and then start to edit it.

In [1]:
##############################################################################################
# If you're using Google Colab, uncomment this section by selecting the whole section and press
# ctrl+'/' on your and keyboard. Run it before you start programming, this will enable the nice 
# LaTeX "display()" function for you. If you're using the local Jupyter environment, leave it alone
##############################################################################################
import sympy as sym
import numpy as np

# def custom_latex_printer(exp,**options):
#     from google.colab.output._publish import javascript
#     url = "https://cdnjs.cloudflare.com/ajax/libs/mathjax/3.1.1/latest.js?config=TeX-AMS_HTML"
#     javascript(url=url)
#     return sym.printing.latex(exp,**options)
# sym.init_printing(use_latex="mathjax",latex_printer=custom_latex_printer)

In [49]:
#helper functions

def SOnAndRnToSEn(R, p):
       
    #do type checking for the matrix types
    if type(R) == list:
        R = np.matrix(R)
        
    n = R.shape[0]
    if ((R.shape[0] != R.shape[1]) or                               #R is NP array or Sym matrix
        ((type(p) is np.ndarray and max(p.shape) != R.shape[0]) or  #p is NP array and shape mismatch or.. 
          ((isinstance(p, list) or isinstance(p, sym.Matrix)) and 
            ( len(p) != R.shape[0] ))   )  ):                       #p is Sym matrix or "list" and shape mismatch
        raise Exception(f"Shape of R {R.shape} and p ({len(p)}) mismatch; exiting.")
        return None
        
    #construct a matrix based on returning a Sympy Matrix
    if isinstance(R, sym.Matrix) or isinstance(p, sym.Matrix): 
        #realistically one of these needs to be symbolic to do this

        if isinstance(R, np.ndarray) or isinstance(p, np.ndarray):
            raise Exception("R and p cannot mix/match Sympy and Numpy types")
            return None
        
        G = sym.zeros(n+1)
        G[:n, n] = sym.Matrix(p)
    
    #construct a matrix based on returning a Numpy matrix
    elif isinstance(R, np.ndarray) or isinstance(R, list):
        G = np.zeros([n+1, n+1])
        print(f"Debug: \n\nR:\n{R}    \n\np:\n{p}   ")
        G[:n, n] = np.array(p).T
        
    else:
        raise Exception("Error: type not recognized")
        return None
    
    G[:n,:n] = R
    G[-1,-1] = 1
    return G  

def SEnToSOnAndRn(SEnmat):
    '''Decomposes a SE(n) vector into its rotation matrix and displacement components.
    '''
    if isinstance(SEnmat, list):
        SEnmat = np.matrix(SEnmat)
    n = SEnmat.shape[0]
    return SEnmat[:(n-1), :(n-1)], SEnmat[:(n-1), n-1]

#test cases

#SO(2) and R2 - numpy
mat1 = np.matrix([[1,2],[3,4]])
p1 = [5,6]
out = SOnAndRnToSEn(mat1, p1)
assert np.array_equal(out,  np.matrix([[1,2,5],[3,4,6],[0,0,1]]) ), f"{out}"

#SO(2) and R2 - sympy
mat2 = sym.Matrix([[5,6],[7,8]])
p2 = [9,0]
out = SOnAndRnToSEn(mat2, p2)
assert out - sym.Matrix([[5,6,9],[7,8,0],[0,0,1]]) == sym.zeros(3,3), f"{out}"

#SO(3) and R3 - numpy 
mat3 = np.matrix([[1,2,3],[4,5,6],[7,8,9]])
p3 = [1.1,2.2,3.3]
out = SOnAndRnToSEn(mat3, p3)
assert np.array_equal(out,  np.matrix([[1,2,3,1.1],[4,5,6,2.2],[7,8,9,3.3],[0,0,0,1]]) ), f"{out}"

#SO(3) and R3 - sympy 
mat4 = sym.Matrix([[1,2,3],[4,5,6],[7,8,9]])
p4 = [4.4,5.5,6.6]
out = SOnAndRnToSEn(mat4, p4)
diff = out - sym.Matrix([[1,2,3,4.4],[4,5,6,5.5],[7,8,9,6.6],[0,0,0,1]])
assert diff == sym.zeros(4,4), f"{out}\n\n{diff}"

#dimensional mismatch - check that it throws an error
#SOnAndRnToSEn(mat2, p4)

#type mismatch - check that it throws an error
#SOnAndRnToSEn(mat2, sym.Matrix(p1))
#SOnAndRnToSEn(mat1, np.matrix(p2))

#SE(3)
SE3mat = SOnAndRnToSEn(np.identity(3), [1,2,3])
[SO3, R3] = SEnToSOnAndRn(SE3mat)
assert np.array_equal(SO3, np.identity(3)) and np.array_equal(R3, [1,2,3]), f"{SO3}\n{R3}"

#SE(2)
SE3mat = SOnAndRnToSEn(np.identity(2), [4,5])
[SO2, R2] = SEnToSOnAndRn(SE3mat)
assert np.array_equal(SO2, np.identity(2)) and np.array_equal(R2, [4,5]), f"{SO2}\n{R2}"

print("All assertions passed")


Debug: 

R:
[[1 2]
 [3 4]]    

p:
[5, 6]   
Debug: 

R:
[[1 2 3]
 [4 5 6]
 [7 8 9]]    

p:
[1.1, 2.2, 3.3]   
Debug: 

R:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]    

p:
[1, 2, 3]   
Debug: 

R:
[[1. 0.]
 [0. 1.]]    

p:
[4, 5]   
All assertions passed


In [43]:
def HatVector3(w):
    '''Turns a vector in R3 to a skew-symmetric matrix in so(3). 
    Works with both Sympy and Numpy matrices.
    '''
     
    if isinstance(w, list):
        w_hat = np.matrix([
            [    0, -w[2],  w[1]],
            [ w[2],     0, -w[0]],
            [-w[1],  w[0],     0]
        ])
        
    else: #NP and Sym
        w_hat = type(w)([
            [    0, -w[2],  w[1]],
            [ w[2],     0, -w[0]],
            [-w[1],  w[0],     0]
        ])

    return w_hat

###

def UnhatMatrix3(w_hat):
    '''Turns a skew-symmetric matrix in so(3) into a vector in R3.
    '''
    if isinstance(w_hat, list):
        w_hat = np.matrix(w_hat)
    
    #NP and Sym
    w = type(w_hat)([
        -w_hat[1,2],
        w_hat[0,2],
        -w_hat[0,1],
    ])
        
    #numpy matrix type has an extra dimension; remove it
    if isinstance(w_hat, np.matrix):
        w = np.asarray(w)[0]
    
    return w

def InvSEn(SEnmat):
    '''Takes the inverse of a SE(n) matrix.
    Compatible with Numpy, Sympy, and list formats.
    '''
    if isinstance(SEnmat, list):
        SEnmat = np.matrix(SEnmat)
    ###
    n = SEnmat.shape[0]
    R = SEnmat[:(n-1), :(n-1)]
    p = SEnmat[:(n-1),   n-1 ]
        
    return SOnAndRnToSEn(R.T, -R.T @ p)
    
###

#testing
w1 = [6,5,4]
print(f"\nHat: \n{HatVector3(w1)}")

w_hat1 = [
    [0, -9, 8],
    [9, 0, -7],
    [-8, 7, 0]
]
print(f"\nUnhat: \n{UnhatMatrix3(w_hat1)}")

T1 = [
    [1,  2,  3,  4],
    [5,  6,  7,  8],
    [9, 10, 11, 12],
    [0,  0,  0,  1]
]

print(f"\nTransInv: \n{InvSEn(T1)}")


Hat: 
[[ 0 -4  5]
 [ 4  0 -6]
 [-5  6  0]]

Unhat: 
[7 8 9]

TransInv: 
[[   1.    5.    9. -152.]
 [   2.    6.   10. -176.]
 [   3.    7.   11. -200.]
 [   0.    0.    0.    1.]]


In [ ]:
def compute_EL(lagrangian, q):
    '''
    Helper function for computing the Euler-Lagrange equations for a given system,
    so I don't have to keep writing it out over and over again.
    
    Inputs:
    - lagrangian: our Lagrangian function in symbolic (Sympy) form
    - q: our state vector [x1, x2, ...], in symbolic (Sympy) form
    
    Outputs:
    - eqn: the Euler-Lagrange equations in Sympy form
    '''
    
    # wrap system states into one vector (in SymPy would be Matrix)
    #q = sym.Matrix([x1, x2])
    qd = q.diff(t)
    qdd = qd.diff(t)

    # compute derivative wrt a vector, method 1
    # wrap the expression into a SymPy Matrix
    L_mat = sym.Matrix([lagrangian])
    dL_dq = L_mat.jacobian(q)
    dL_dqdot = L_mat.jacobian(qd)

    #set up the Euler-Lagrange equations
    LHS = dL_dq - dL_dqdot.diff(t)
    RHS = sym.zeros(1, len(q))
    eqn = sym.Eq(LHS.T, RHS.T)
    
    return eqn

def solve_EL(eqn, var):
    '''
    Helper function to solve and display the solution for the Euler-Lagrange
    equations.
    
    Inputs:
    - eqn: Euler-Lagrange equation (type: Sympy Equation())
    - var: state vector (type: Sympy Matrix). typically a form of q-doubledot
        but may have different terms
    
    Outputs:
    - Prints symbolic solutions
    - Returns symbolic solutions in a dictionary
    '''
    
    soln = sym.solve(eqn, var, dict = True)
    eqns_solved = []
    
    
    for i, sol in enumerate(soln):
        for x in list(sol.keys()):
            eqn_solved = sym.Eq(x, sol[x])
            eqns_solved.append(eqn_solved)
            
    return eqns_solved

def solve_EL_hw7(lamb_list, phi_list, F_mat, q, lhs):
    """This version of the Euler-Lagrange equations uses both a constraint 
    """
    
    qd = q.diff(t)
    qdd = qd.diff(t)
    expr_matrix = lhs - F_mat
    phidd_matrix = sym.Matrix([0])
    
    #add constraint terms to matrix of expressions to solve
    for i in range(len(phi_list)):
        phi = phi_list[i]
        lamb = lamb_list[i]
        
        phidd = phi.diff(t).diff(t)
        lamb_grad = sym.Matrix([lamb * phi.diff(a) for a in q])
        qdd = qdd.row_insert(len(qdd, sym.Matrix([lamb]))
    
        #format equations so they're all in one matrix
        expr_matrix = expr_matrix - lamb_grad
        phidd_matrix = sym.Matrix([phidd])
        expr_matrix = expr_matrix.row_insert(len(expr_matrix), phidd_matrix)
    
    print("Equations to be solved (LHS - lambda * grad(phi) = 0):")
    RHS = sym.zeros(len(expr_matrix), 1)
    disp_eq = sym.Eq(expr_matrix, RHS)
    display(disp_eq)
    print("Variables to solve for:")
    display(qdd)

    #solve E-L equations
    eqns_solved = solve_EL(expr_matrix, qdd)
    return eqns_solved


In [ ]:
def rk4(dxdt, x, t, dt):
    '''
    Applies the Runge-Kutta method, 4th order, to a sample function,
    for a given state q0, for a given step size. Currently only
    configured for a 2-variable dependent system (x,y).
    ==========
    dxdt: a Sympy function that specifies the derivative of the system of interest
    t: the current timestep of the simulation
    x: current value of the state vector
    dt: the amount to increment by for Runge-Kutta
    ======
    returns:
    x_new: value of the state vector at the next timestep
    '''      
    k1 = dt * dxdt(t, x)
    k2 = dt * dxdt(t + dt/2.0, x + k1/2.0)
    k3 = dt * dxdt(t + dt/2.0, x + k2/2.0)
    k4 = dt * dxdt(t + dt, x + k3)
    x_new = x + (k1 + 2.0*k2 + 2.0*k3 + k4)/6.0
    
    return x_new


def simulate(f, x0, tspan, dt, integrate):
    """
    This function takes in an initial condition x0, a timestep dt,
    a time span tspan consisting of a list [min_time, max_time],
    as well as a dynamical system f(x) that outputs a vector of the
    same dimension as x0. It outputs a full trajectory simulated
    over the time span of dimensions (xvec_size, time_vec_size).
    
    Parameters
    ============
    f: Python function
        derivate of the system at a given step x(t), 
        it can considered as \dot{x}(t) = func(x(t))
    x0: NumPy array
        initial conditions
    tspan: Python list
        tspan = [min_time, max_time], it defines the start and end
        time of simulation
    dt:
        time step for numerical integration
    integrate: Python function
        numerical integration method used in this simulation

    Return
    ============
    x_traj:
        simulated trajectory of x(t) from t=0 to tf
    """
    N = int((max(tspan)-min(tspan))/dt)
    x = np.copy(x0)
    tvec = np.linspace(min(tspan),max(tspan),N)
    xtraj = np.zeros((len(x0),N))
    
    for i in range(N):
        t = tvec[i]
        xtraj[:,i]=integrate(f,x,t,dt)
        x = np.copy(xtraj[:,i])
    return xtraj 
      

## Problem 1 (20pts)

Show that if $R\in SO(n)$, then the matrix $A=\frac{d}{dt}(R)R^{-1}$ is skew symmetric.
    
**Turn in: A scanned (or photograph from your phone or webcam) copy of your hand written solution. Or you can use \LaTeX.**

## Problem 2 (20pts)

Show that $\underline{\widehat{\omega}}\,\underline{r}_b= -\underline{\widehat{r}}_b\, \underline{\omega}$. 
    
**Turn in: A scanned (or photograph from your phone or webcam) copy of your hand written solution. Or you can use \LaTeX.**

In [54]:
from IPython.core.display import HTML
display(HTML("<table><tr><td><img src='https://github.com/MuchenSun/ME314pngs/raw/master/biped_simplified.jpg' width=600' height='350'></table>"))

## Problem 3 (60pts)

Consider a person doing the splits (shown in the image above). To simplify the model, we ignore the upper body and assume the knees can not bend --- which means we only need four variables $q=[x, y, \theta_1, \theta_2]$ to configure the system. $x$ and $y$ are the position of the intersection point of the two legs, $\theta_1$ and $\theta_2$ are the angles between the legs and the green vertical dash line. The feet are constrained on the ground, and there is no friction between the feet and the ground. 
    
Each leg is a rigid body with length $L=1$, width $W=0.2$, mass $m=1$, and rotational inertia $J=1$ (assuming the center of mass is at the center of geometry). Moreover, there are two torques applied on $\theta_1$ and $\theta_2$ to control the legs to track a desired trajectory:
$$
\begin{aligned}
\theta_1^d(t) & = \frac{\pi}{15} + \frac{\pi}{3} \sin^2(\frac{t}{2}) \\
\theta_2^d(t) & =-\frac{\pi}{15} - \frac{\pi}{3} \sin^2(\frac{t}{2})
\end{aligned}
$$ and the torques are:
$$
\begin{aligned}
F_{\theta_1 }& = -k_1(\theta_1-\theta_1^d)\\
F_{\theta_2} & = -k_1(\theta_2-\theta_2^d)
\end{aligned}
$$ In this problem we use $k_1=20$. 

Given the model description above, define the frames that you need (several example frames are shown in the image as well), simulate the motion of the biped from rest for $t\in[0,10], dt=0.01$, with initial condition $q=[0, L_1\cos(\frac{\pi}{15}), \frac{\pi}{15}, -\frac{\pi}{15}]$. You will need to modify the animation function to display the leg as a rectangle, an example of the animation can be found at [https://youtu.be/w8XHYrEoWTc](https://youtu.be/w8XHYrEoWTc). 

*Hint 1: Even though this is a 2D system, in order to compute kinetic energy from both translation and rotation you will need to model the system in the 3D world --- the $z$ coordinate is always zero and the rotation is around the $z$ axis (based on these facts, what should the SE(3) matrix and rotational inertia tensor look like?). This also means you need to represent transformations in  SE(3) and the body velocity $\mathcal{V}_b\in\mathbb{R}^6$.*

*Hint 2: It could be helpful to define several helper functions for all the matrix operations you will need to use. For example, a function that returns SE(3) matrices given a rotation angle and 2D translation vector, functions for ''hat'' and ''unhat'' operations, a function for the matrix inverse of SE(3) (which is definitely not the same as the SymPy matrix inverse function), and a function that returns the time derivative of SO(3) or SE(3).*

*Hint 3: In this problem the external force depends on time $t$. Therefore, in order to solve for the symbolic solution you need to substitute your configuration variables, which are defined as symbolic functions of time $t$ (such as $\theta_1(t)$ and $\frac{d}{dt}\theta_1(t)$), with dummy symbolic variables. For the same reason (the dynamics now explicitly depend on time), you will need to do some tiny modifications on the ''integrate'' and ''simulate'' functions, a good reference can be found at [https://en.wikipedia.org/wiki/Runge-Kutta_methods](https://en.wikipedia.org/wiki/Runge-Kutta_methods).*

*Hint 4: Symbolically solving this system should be fast, but if you encountered some problem when solving the dynamics symbolically, an alternative method is to solve the system numerically --- substitute in the system state at each time step during simulation and solve for the numerical solution --- but based on my experience, this would cost more than one hour for 500 time steps, so it's not recommended.*

*Hint 5: The animation of this problem is similar to the one in last homework --- the coordinates of the vertices in the body frame are constant, you just need to transfer them back to the world frame using the the transformation matrices you already have in the simulation.*

*Hint 6: Be careful to consider the relationship between the frames and to not build in any implicit assumptions (such as assuming some variables are fixed).*

*Hint 7: The rotation, by convention, is assumed to follow the right hand rule, which means the z-axis is out of the screen and the positive rotation orientation is counter-clockwise. Make sure you follow a consistent set of positive directions for all the computation.*

*Hint 8: This problem is designed as a ''mini-project'', it could help you estimate the complexity of your final project, and you could adjust your proposal based on your experience with this problem.*

**Turn in: A copy of the code used to simulate and animate the system. Also, include a plot of the trajectory and upload a video of the animation separately through Canvas. The video should be in ".mp4" format, you can use screen capture or record the screen directly with your phone.**

In [ ]:
## Your code goes here

In [66]:
#define variables
L1, L2, m, J, W, g = sym.symbols(r'L_1, L_2, m, J, W, g')
t = sym.symbols(r't')
x = sym.Function(r'x')(t)
y = sym.Function(r'y')(t)
theta1 = sym.Function(r'\theta_1')(t)
theta2 = sym.Function(r'\theta_2')(t)

q = sym.Matrix([x, y, theta1, theta2])
qd = q.diff(t)
qdd = qd.diff(t)

#not included: xd, yd, ..., q_ext

#define transformation matrices. let A1 be in the direction of 
#the right leg and A2 be in the direction of the left leg

#------right leg------#
Raa1 = sym.Matrix([
    [sym.cos(theta1), -sym.sin(theta1), 0],
    [sym.sin(theta1),  sym.cos(theta1), 0],
    [              0,                0, 1]
])

Rdf = sym.Matrix([
    [sym.cos(-theta1), -sym.sin(-theta1), 0],
    [sym.sin(-theta1),  sym.cos(-theta1), 0],
    [              0,                0,   1]
])

p_a1b = sym.Matrix([0, -L1/2, 0])
p_bd =  sym.Matrix([0, -L1/2, 0])

Gaa1 = SOnAndRnToSEn(Raa1, [0,0,0]) 
Ga1b = SOnAndRnToSEn(sym.eye(3), p_a1b)
Gbd  = SOnAndRnToSEn(sym.eye(3),  p_bd) 
Gdf  = SOnAndRnToSEn(Rdf,  [0,0,0]) 

#------left leg------#

Raa2 = sym.Matrix([
    [sym.cos(-theta2), -sym.sin(-theta2), 0],
    [sym.sin(-theta2),  sym.cos(-theta2), 0],
    [              0,                0,   1]

])

Reg = sym.Matrix([
    [sym.cos(theta2), -sym.sin(theta2), 0],
    [sym.sin(theta2),  sym.cos(theta2), 0],
    [              0,                0, 1]
])

p_a2c = sym.Matrix([0, -L2/2, 0])
p_ce =  sym.Matrix([0, -L2/2, 0])

Gaa2 = SOnAndRnToSEn(Raa2, [0,0,0])
Ga2c = SOnAndRnToSEn(sym.eye(3), p_a2c)
Gce  = SOnAndRnToSEn(sym.eye(3), p_ce)
Geg  = SOnAndRnToSEn(Reg, [0,0,0])

#---------------------#

#combine transformation matrices
Gsa = SOnAndRnToSEn(sym.eye(3),sym.Matrix([x,y,0]))
Gsb = Gsa @ Gaa1 @ Ga1b
Gsc = Gsa @ Gaa2 @ Ga2c
Gsd = Gsb @ Gbd
Gse = Gsc @ Gce
Gsf = Gsd @ Gdf
Gsg = Gse @ Geg

#define important positions in space
posn_CM2 = Gsc @ sym.Matrix([0,0,0,1]) #"bar" version of 3D posn
posn_CM1 = Gsb @ sym.Matrix([0,0,0,1])

bottom1 = Gsg @ sym.Matrix([0,0,0,1])
bottom2 = Gsf @ sym.Matrix([0,0,0,1])

print("bottom1 (bar):")
display(bottom1)
print("bottom2 (bar):")
display(bottom2)


bottom1 (bar):


Matrix([
[-L_2*sin(\theta_2(t)) + x(t)],
[-L_2*cos(\theta_2(t)) + y(t)],
[                           0],
[                           1]])

bottom2 (bar):


Matrix([
[ L_1*sin(\theta_1(t)) + x(t)],
[-L_1*cos(\theta_1(t)) + y(t)],
[                           0],
[                           1]])

In [77]:
#define lagrangian
ybottom1 = bottom1[1] 
ybottom2 = bottom2[1]
y_CM1    = posn_CM1[1]
y_CM2    = posn_CM2[1]

U = m*g* (y_CM1 + y_CM2)

#to define kinetic energy: need axis of rotation in the body's own frame; the z axis
#KE_rot1 = 0.5*w.T * I * w = 0.5(I)(1)(thetand)^2
ti = sym.symbols(r't_i')
k1 = sym.symbols(r'k_1')
theta1_des =  sym.pi/15 + sym.pi/3 * (sym.sin(t/2))**2 #des = "desired"
theta1d_des = theta1_des.diff(t).simplify()
theta2_des = -sym.pi/15 - sym.pi/3 * (sym.sin(t/2))**2
theta2d_des = theta2_des.diff(t).simplify()

F1 = -k1 * (theta1 - theta1_des)
F2 = -k1 * (theta2 - theta2_des)

# display(F1)
# display(F2)

KE1 = (0.5 * sym.Matrix([0,0,1]).T * J * sym.Matrix([0,0,1]) * (theta1d_des)**2)[0]
KE2 = (0.5 * sym.Matrix([0,0,1]).T * J * sym.Matrix([0,0,1]) * (theta2d_des)**2)[0]
lagrangian = KE1 + KE2 + U

display(KE1)
display(KE2)
display(lagrangian)

0.0138888888888889*pi**2*J*sin(t)**2

0.0138888888888889*pi**2*J*sin(t)**2

0.0277777777777778*pi**2*J*sin(t)**2 + g*m*(-L_1*cos(\theta_1(t))/2 - L_2*cos(\theta_2(t))/2 + 2*y(t))

In [ ]:
#set up the left and right hand sides of Euler-Lagrange EQs